# 🐘 Exportação de Tabelas do PostgreSQL para o S3 (Camada Raw)


Este notebook conecta-se ao banco de dados PostgreSQL e exporta as tabelas:
- `clientes`
- `produtos`
- `pedidos`
- `itens_pedido`
- `tipos_produto`

Os dados são salvos na camada **raw** de um bucket S3 no formato `.csv`.


In [ ]:
%pip install psycopg2-binary pandas boto3

In [10]:
import pandas as pd
import psycopg2
import boto3
from io import StringIO
from botocore.exceptions import ClientError

# === CONFIGURAÇÕES ===

# PostgreSQL
pg_config = {
    "host": "postgres-db.cu4mvwwdzs1u.us-east-1.rds.amazonaws.com",
    "database": "db_relacional",
    "user": "postgres",
    "password": "Fiap#2025",
    "port": 5432
}

# AWS S3
bucket_name = "aula-data-lake"
s3_prefix = "raw/"

# Tabelas a exportar
tabelas = ['clientes', 'produtos', 'pedidos', 'itens_pedido', 'tipos_produto']

# === CONEXÃO COM POSTGRES ===

conn = psycopg2.connect(**pg_config)
print("Conexão com o PostgreSQL estabelecida.")

# === CONEXÃO COM S3 ===
s3 = boto3.client('s3')
region = s3.meta.region_name or "us-east-1"  # região do client S3

# === VERIFICAR/CRIAR BUCKET ===
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe.")
except ClientError as e:
    error_code = int(e.response['Error']['Code'])
    if error_code == 404:
        print(f"Bucket '{bucket_name}' não existe. Criando...")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso.")
    else:
        raise

# === EXPORTAÇÃO ===
for tabela in tabelas:
    print(f"Exportando tabela: {tabela}")
    df = pd.read_sql(f"SELECT * FROM {tabela};", conn)

    # Salvar como CSV em memória
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    # Enviar para S3
    s3_key = f"{s3_prefix}{tabela}.csv"
    s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())
    print(f"✅ {tabela} salva no S3 em: s3://{bucket_name}/{s3_key}")

# === FECHAR CONEXÃO ===
conn.close()
print("Exportação concluída com sucesso.")


Conexão com o PostgreSQL estabelecida.
Bucket 'aula-data-lake' já existe.
Exportando tabela: clientes


/var/folders/71/k_krjbt51f5106pgp409lg0m0000gn/T/ipykernel_86005/1455349903.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {tabela};", conn)


✅ clientes salva no S3 em: s3://aula-data-lake/raw/clientes.csv
Exportando tabela: produtos
✅ produtos salva no S3 em: s3://aula-data-lake/raw/produtos.csv
Exportando tabela: pedidos
✅ pedidos salva no S3 em: s3://aula-data-lake/raw/pedidos.csv
Exportando tabela: itens_pedido
✅ itens_pedido salva no S3 em: s3://aula-data-lake/raw/itens_pedido.csv
Exportando tabela: tipos_produto
✅ tipos_produto salva no S3 em: s3://aula-data-lake/raw/tipos_produto.csv
Exportação concluída com sucesso.


# 🐘 Transferencia de arquivos CSV para as camadas do LakeHouse (Camada Silver e Gold)

In [4]:
%pip install pandas pyarrow fsspec s3fs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 9.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 24.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: yarl
    Found existing installation: yarl 1.9.2
    Uninstalling yarl-1.9.2:
      Successfully uninstalled yarl-1.9.2
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.106
    Uninstalling botocore-1.34.106:
      Successfully uninstalled botocore-1.34.106
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.4
    Uninstalling aiohttp-3.8.4:
      Successfully uninstalled aiohttp-3.8.4
ERROR: pip's dependency 

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

# Para acessar S3, Pandas usa o s3fs automaticamente
RAW_PREFIX = 's3://aula-data-lake/raw/'
SILVER_PREFIX = 's3://aula-data-lake/silver/'
GOLD_PREFIX = 's3://aula-data-lake/gold/'

# 1. Silver Layer: Leitura e Limpeza dos Dados
clientes = pd.read_csv(RAW_PREFIX + 'clientes.csv')
produtos = pd.read_csv(RAW_PREFIX + 'produtos.csv')
tipos_produto = pd.read_csv(RAW_PREFIX + 'tipos_produto.csv')
pedidos = pd.read_csv(RAW_PREFIX + 'pedidos.csv')
itens_pedido = pd.read_csv(RAW_PREFIX + 'itens_pedido.csv')

# Padronizar colunas para lower case
for df in [clientes, produtos, tipos_produto, pedidos, itens_pedido]:
    df.columns = [col.lower() for col in df.columns]

# Limpeza básica
clientes = clientes.drop_duplicates().dropna(subset=['id_cliente'])
produtos = produtos.drop_duplicates().dropna(subset=['id_produto'])
tipos_produto = tipos_produto.drop_duplicates().dropna(subset=['id_tipo'])
pedidos = pedidos.drop_duplicates().dropna(subset=['id_pedido', 'id_cliente'])
itens_pedido = itens_pedido.drop_duplicates().dropna(subset=['id_item', 'id_pedido', 'id_produto'])

# Padronização de datas
if 'data_pedido' in pedidos.columns:
    pedidos['data_pedido'] = pd.to_datetime(pedidos['data_pedido'], errors='coerce')

# Salvar camada Silver em Parquet no S3
clientes.to_parquet(SILVER_PREFIX + 'clientes/', index=False)
produtos.to_parquet(SILVER_PREFIX + 'produtos/', index=False)
tipos_produto.to_parquet(SILVER_PREFIX + 'tipos_produto/', index=False)
pedidos.to_parquet(SILVER_PREFIX + 'pedidos/', index=False)
itens_pedido.to_parquet(SILVER_PREFIX + 'itens_pedido/', index=False)

# 2. Gold Layer: Enriquecimento e Agregações

# Fato de vendas com todas as dimensões
gold_vendas = itens_pedido.merge(pedidos, on='id_pedido') \
    .merge(produtos, on='id_produto') \
    .merge(tipos_produto, on='id_tipo') \
    .merge(clientes, on='id_cliente')

# Valor total do item
if 'quantidade' in gold_vendas.columns and 'preco_unitario' in gold_vendas.columns:
    gold_vendas['valor_total_item'] = gold_vendas['quantidade'] * gold_vendas['preco_unitario']

# Criação do campo anomesdia
if 'data_pedido' in gold_vendas.columns:
    gold_vendas['anomesdia'] = gold_vendas['data_pedido'].dt.strftime('%Y%m%d')

# Exemplo: Resumo por cliente
gold_vendas_por_cliente = gold_vendas.groupby(['id_cliente', 'nome']) \
    .agg({'valor_total_item': 'sum', 'id_pedido': 'nunique'}) \
    .rename(columns={'valor_total_item': 'valor_total_comprado', 'id_pedido': 'num_pedidos'}) \
    .reset_index()

# Exemplo: Resumo por tipo de produto
gold_vendas_por_tipo = gold_vendas.groupby(['id_tipo', 'nome_tipo']) \
    .agg({'valor_total_item': 'sum', 'quantidade': 'sum'}) \
    .rename(columns={'valor_total_item': 'total_vendido', 'quantidade': 'quantidade_total'}) \
    .reset_index()

# Salvar camada Gold no S3, particionando por anomesdia
gold_vendas.to_parquet(GOLD_PREFIX + 'fato_vendas/', partition_cols=['anomesdia'], index=False)
gold_vendas_por_cliente.to_parquet(GOLD_PREFIX + 'vendas_por_cliente/', index=False)
gold_vendas_por_tipo.to_parquet(GOLD_PREFIX + 'vendas_por_tipo/', index=False)

print("Processamento das camadas Silver e Gold no S3 particionado por anomesdia finalizado com sucesso!")


Processamento das camadas Silver e Gold no S3 particionado por anomesdia finalizado com sucesso!
